# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers sentence-transformers torchmetrics gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import warnings
warnings.filterwarnings("ignore")

# Import

In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from copy import deepcopy
from torchmetrics import Precision
import torch
import json
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# Change label matching

In [6]:
def process(text):
    return text.strip()

In [7]:
with open("/content/drive/MyDrive/Achatbot/data/new2oldmatch.json", 'r') as f:
    new2oldmatch = json.load(f)
    new2oldmatch = {process(u.lower()): process(v) for u, v in new2oldmatch.items()}
    old2newmatch = {u: v for v, u in new2oldmatch.items()}

# Dataset

In [8]:
train_data = pd.read_csv("/content/drive/MyDrive/Achatbot/data/train_valid_test_4/train.csv")
train_data.rename(columns={'Unnamed: 0': "ID"}, inplace=True)
train_data['Câu lệnh có sẵn']= train_data['Câu lệnh có sẵn'].apply(lambda x: old2newmatch[process(x)])
train_data.head()

,ID,Câu lệnh sinh ra,Câu lệnh có sẵn
0,1587,đọc kỹ các hd sử dụng,mở hướng dẫn sử dụng
1,526,xin mở cảnh báo nhập kế hoạch trong năm để có ...,mở cảnh báo nhập kế hoạch năm
2,2718,xin hãy mở màn hình trình diễn để hiển thị báo...,mở màn hình trình chiếu báo cáo tháng
3,144,mở danh mục tổng quan sẵn sàng và không để chậ...,mở danh mục chung
4,921,mở kết quả thống kê liên ngành tháng trước,mở kết quả liên ngành (tháng n-1)


In [9]:
train_false_data = pd.read_csv("/content/drive/MyDrive/Achatbot/data/train_valid_test_4/train_false.csv")
train_false_data.rename(columns={'Unnamed: 0': "ID"}, inplace=True)
train_false_data['Câu lệnh có sẵn'] = train_false_data['Câu lệnh có sẵn'].apply(lambda x: old2newmatch[process(x)])
train_false_data.head()

,ID,Câu lệnh sinh ra,Câu lệnh có sẵn
0,0,đọc kỹ các hd sử dụng,mở quản lý menu
1,1,xin mở cảnh báo nhập kế hoạch trong năm để có ...,mở báo cáo văn phòng tập đoàn
2,2,xin hãy mở màn hình trình diễn để hiển thị báo...,mở danh mục chung
3,3,mở danh mục tổng quan sẵn sàng và không để chậ...,mở dữ liệu kế hoạch năm
4,4,mở kết quả thống kê liên ngành tháng trước,mở dữ liệu thực hiện ngày


In [10]:
valid_data = pd.read_csv("/content/drive/MyDrive/Achatbot/data/train_valid_test_4/valid.csv")
valid_data.rename(columns={'Unnamed: 0': "ID"}, inplace=True)
valid_data['Câu lệnh có sẵn'] = valid_data['Câu lệnh có sẵn'].apply(lambda x: old2newmatch[process(x)])
valid_data.head()

,ID,Câu lệnh sinh ra,Câu lệnh có sẵn
0,1663,xin mở dashboard,mở dashboard
1,886,hãy xem công thức ct kỹ thuật,mở công thức chỉ tiêu
2,868,có báo cáo công thức chỉ tiêu nào không?,mở công thức chỉ tiêu
3,2402,vui lòng mở tra cứu để biết tần suất sử dụng h...,mở tra cứu tần suất sử dụng hệ thống
4,2057,đảm bảo rằng quản lý đa ngôn ngữ đã được bật,mở quản lý đa ngôn ngữ


In [11]:
test_data = pd.read_csv("/content/drive/MyDrive/Achatbot/data/train_valid_test_4/test.csv")
test_data.rename(columns={'Unnamed: 0': "ID"}, inplace=True)
test_data['Câu lệnh có sẵn'] = test_data['Câu lệnh có sẵn'].apply(lambda x: old2newmatch[process(x)])
test_data.head()

,ID,Câu lệnh sinh ra,Câu lệnh có sẵn
0,578,bạn hãy mở lên báo cáo dl kế hoạch tháng,mở dữ liệu kế hoạch tháng
1,2181,bắt đầu cài đặt quyền quản lý người nhận cb ng...,mở quản lý cấu hình người nhận cảnh báo
2,2020,truy cập vào cài đặt quản lý cấu hình truy vấn,mở quản lý cấu hình truy vấn
3,333,hãy mở quá trình quản lý chu kỳ nhập tài chính...,"mở quản lý chu kỳ import tài chính, kinh doanh"
4,1515,mong bạn mở bc kpi cđ để đưa ra biện pháp,mở báo cáo kpi chấm điểm


In [57]:
testcase_data = pd.read_csv("/content/drive/MyDrive/Achatbot/data/test_case/processed_test_case.csv")
testcase_data['Câu lệnh nhập vào'] = testcase_data['Câu lệnh nhập vào'].apply(lambda x: x.lower())
testcase_data.rename(columns={'Câu lệnh nhập vào': 'Câu lệnh sinh ra'}, inplace=True)
testcase_data['Câu lệnh có sẵn'] = testcase_data['Câu lệnh có sẵn'].apply(lambda x: old2newmatch[process(x)])
testcase_data.drop(columns=['Unnamed: 0', 'Output', 'Dự đoán'], inplace=True)
testcase_data

,Câu lệnh sinh ra,Câu lệnh có sẵn
0,mở cảnh báo nhập kh tháng,mở cảnh báo nhập kế hoạch tháng
1,cảnh báo nhập kh tháng,mở cảnh báo nhập kế hoạch tháng
2,cảnh báo kh tháng,mở cảnh báo nhập kế hoạch tháng
3,cảnh báo tháng,mở cảnh báo nhập kế hoạch tháng
4,cảnh báo kế hoạch tháng,mở cảnh báo nhập kế hoạch tháng
...,...,...
285,bc kpi,mở báo cáo kpi chấm điểm
286,bc kpi,mở báo cáo kpi chấm điểm
287,bc kpi cđ,mở báo cáo kpi chấm điểm
288,mở báo cáo kpi chấm điểm,mở báo cáo kpi chấm điểm


# Model

In [12]:
# model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2',
#                             cache_folder="/content/drive/MyDrive/Achatbot/model_dir")

In [13]:
# model.save("/content/drive/MyDrive/Achatbot/output/model")

In [14]:
# model.load("/content/drive/MyDrive/Achatbot/output/model")

In [15]:
def cosinesimilarity(vector1, vector2):
    cosine = np.dot(vector1, vector2)/(norm(vector1)*norm(vector2))
    return cosine

def encode_all_and_match(input, db_in):
    print("Input:", input)
    embedding = model.encode(input)
    df = pd.DataFrame(list(zip(db_in, [0]*len(db_in))), columns=["Câu lệnh có sẵn", "Điểm"])
    for i, func in tqdm(enumerate(db_in)):
        embedding2 = model.encode(func)
        df['Điểm'].loc[i] = cosinesimilarity(embedding, embedding2)
    df.sort_values(by=['Điểm'], inplace=True, ascending=False)
    return df

def encode_database(db_in, show_text=True):
    df = pd.DataFrame(list(zip(db_in, [[]]*len(db_in))), columns=["Câu lệnh có sẵn", "Embedding"])
    for i, func in enumerate(db_in):
        embedding2 = model.encode(func)
        df['Embedding'].loc[i] = embedding2
    else:
        if show_text:
            print()
            print("Encode database successfully")
    return df

def encode_input_and_match(input_in, db_dff):
    print("Input:", input_in)
    embedding = model.encode(input_in)
    scores = []
    db_df_in = deepcopy(db_dff)
    db_in = list(set(db_df_in['Câu lệnh có sẵn'].tolist()))
    for i, func in tqdm(enumerate(db_in)):
        embedding2 = db_df_in['Embedding'].loc[i]
        scores.append(cosinesimilarity(embedding, embedding2))
    else:
        print()
        print("Matching process finished.")
    db_df_in["Điểm"] = scores
    db_df_in.sort_values(by=['Điểm'], inplace=True, ascending=False)
    return db_df_in[["Câu lệnh có sẵn", "Điểm"]]

def encode_input_and_return_output(input_in, db_dff):
    embed1 = model.encode(input_in)
    scores = []
    db_df_in = deepcopy(db_dff)
    db_in = list(set(db_df_in['Câu lệnh có sẵn'].tolist()))
    for i, func in enumerate(db_in):
        embed2 = db_df_in['Embedding'].loc[i]
        scores.append(cosinesimilarity(embed1, embed2))
    db_df_in["Điểm"] = scores
    return db_df_in["Câu lệnh có sẵn"].loc[db_df_in['Điểm'].idxmax()]

def encode_input_and_return_top_n(input_in, db_dff, top_k):
    embed1 = model.encode(input_in)
    scores = []
    db_df_in = deepcopy(db_dff)
    db_in = list(set(db_df_in['Câu lệnh có sẵn'].tolist()))
    for i, func in enumerate(db_in):
        embed2 = db_df_in['Embedding'].loc[i]
        scores.append(round(cosinesimilarity(embed1, embed2), 3))
    db_df_in["Điểm"] = scores
    db_df_in.sort_values(by=['Điểm'], inplace=True, ascending=False)
    ids = db_df_in[:top_k].index.tolist()
    output = {db_df_in['Câu lệnh có sẵn'][i]: round(db_df_in['Điểm'][i].item(), 2) for i in ids}
    return output

# Finetune

In [16]:
CSKH_data = []
for i in range(len(train_data)):
    k = train_data["Câu lệnh có sẵn"].loc[i]
    if "CSKH" in k:
        CSKH_data.append(i)

CSKH_data[:5]

[]

# Necessary def

In [17]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

# model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2',
#                             cache_folder="/content/drive/MyDrive/Achatbot/model_dir")
train_examples = []
for i in range(len(train_data)):
    train_examples.append(InputExample(texts=[
        train_data["Câu lệnh sinh ra"].loc[i],
        train_data["Câu lệnh có sẵn"].loc[i]
    ], label=1.0))

for i in range(len(train_false_data)):
    train_examples.append(InputExample(texts=[
        train_false_data["Câu lệnh sinh ra"].loc[i],
        train_false_data["Câu lệnh có sẵn"].loc[i]
    ], label=0.0))
"""
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
train_loss = losses.CosineSimilarityLoss(model=model)
evaluator = EmbeddingSimilarityEvaluator(sentences1=valid_data["Câu lệnh sinh ra"],
                                         sentences2=valid_data["Câu lệnh có sẵn"],
                                         scores=len(valid_data)*[1],
                                         write_csv=True)
"""

'\ntrain_dataset = SentencesDataset(train_examples, model)\ntrain_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)\ntrain_loss = losses.CosineSimilarityLoss(model=model)\nevaluator = EmbeddingSimilarityEvaluator(sentences1=valid_data["Câu lệnh sinh ra"],\n                                         sentences2=valid_data["Câu lệnh có sẵn"],\n                                         scores=len(valid_data)*[1],\n                                         write_csv=True)\n'

In [18]:
for i in range(len(valid_data)):
    k = valid_data["Câu lệnh có sẵn"].loc[i]
    if "chăm sóc" in k:
        print(k)
        hi = i
        break

mở báo cáo chăm sóc khách hàng


In [21]:
input = valid_data["Câu lệnh sinh ra"].loc[hi]
input

'xin mở bc chăm sóc khách hàng'

In [22]:
compared = valid_data["Câu lệnh có sẵn"].loc[hi]
compared

'mở báo cáo chăm sóc khách hàng'

In [23]:
def cosinesimilarity(vector1, vector2):
    cosine = np.dot(vector1, vector2)/(norm(vector1)*norm(vector2))
    return cosine

model = SentenceTransformer("/content/drive/MyDrive/Achatbot/output/model")
model.eval()
embedding = model.encode(input)
embedding2 = model.encode(compared)
print(cosinesimilarity(embedding, embedding2))

0.9786167


In [ ]:
# model = SentenceTransformer("/content/drive/MyDrive/Achatbot/output/model")
# model.load("/content/drive/MyDrive/Achatbot/model_dir/sentence-transformers_distiluse-base-multilingual-cased-v2")

In [ ]:
# model.save("/content/drive/MyDrive/Achatbot/output/model")

In [24]:
database = [cmd for cmd in new2oldmatch.keys()]

def evaluate(modell, datata, epoch, db):
    db_df = encode_database(db, show_text=False)
    diction = {k: v for v, k in enumerate(db)}
    resultsss = deepcopy(datata)
    resultsss['Dự đoán myself'] = [0]*len(resultsss)
    for i, ints in enumerate(resultsss['Câu lệnh sinh ra']):
        output = encode_input_and_return_output(ints, db_df)
        resultsss['Dự đoán myself'].loc[i] = output
    preds = torch.tensor(resultsss['Dự đoán myself'].apply(lambda x: diction[process(x)]).tolist())
    tags = torch.tensor(resultsss['Câu lệnh có sẵn'].apply(lambda x: diction[process(x)]).tolist())
    precision = Precision(task="multiclass", average='micro', num_classes=len(diction))
    result = round(precision(preds, tags).item()*100, 2)
    print(f"Epoch {epoch}: Độ chính xác (Precision) của mô hình: {result}")
    return result

In [35]:
def cosinesimilarity(vector1, vector2):
    cosine = np.dot(vector1, vector2)/(norm(vector1)*norm(vector2))
    return cosine

# model = SentenceTransformer("/content/drive/MyDrive/Achatbot/output/model")
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
train_loss = losses.CosineSimilarityLoss(model=model)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

for i in range(100):
    model.train()
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        show_progress_bar=False
    )
    if i % 10 == 0:
        model.train(False)
        result = evaluate(model, valid_data, i, database)

# model.save("/content/drive/MyDrive/Achatbot/output/model")
# model.load("/content/drive/MyDrive/Achatbot/output/model")

Epoch 0: Độ chính xác (Precision) của mô hình: 94.08
Epoch 10: Độ chính xác (Precision) của mô hình: 94.08
Epoch 20: Độ chính xác (Precision) của mô hình: 94.08
Epoch 30: Độ chính xác (Precision) của mô hình: 94.08
Epoch 40: Độ chính xác (Precision) của mô hình: 94.08
Epoch 50: Độ chính xác (Precision) của mô hình: 94.08
Epoch 60: Độ chính xác (Precision) của mô hình: 94.08
Epoch 70: Độ chính xác (Precision) của mô hình: 94.08
Epoch 80: Độ chính xác (Precision) của mô hình: 94.08
Epoch 90: Độ chính xác (Precision) của mô hình: 94.08


In [58]:
model.train(False)
result = evaluate(model, testcase_data, 0, database)

Epoch 0: Độ chính xác (Precision) của mô hình: 76.55


In [67]:
# model.save("/content/drive/MyDrive/Achatbot/output/model")

# Function Database (List of function commands)

# Test similarity

In [59]:
input = "Một con vịt"
embedding = model.encode(input)
compared = "Hai con lợn"
embedding2 = model.encode(compared)
cosinesimilarity(embedding, embedding2)

0.543596

# Matching a input with database

## Encode database

In [60]:
db_df = encode_database(database)


Encode database successfully


## Encode input and compare embedding with database

In [61]:
df = encode_input_and_match(input, db_df)
df[:5]

Input: Một con vịt


55it [00:00, 16028.82it/s]


Matching process finished.


,Câu lệnh có sẵn,Điểm
38,mở báo cáo chăm sóc khách hàng,0.409024
13,mở báo cáo kpi chấm điểm,0.361756
27,mở dashboard,0.232873
29,mở quản lý báo cáo,0.224407
52,mở báo cáo văn phòng tập đoàn,0.222905


# Dự đoán trên tập dữ liệu

In [62]:
datadata = testcase_data
# datadata = pd.read_csv("/content/drive/MyDrive/Achatbot/data/train_valid_test/train.csv")
datadata

,Câu lệnh sinh ra,Câu lệnh có sẵn
0,mở cảnh báo nhập kh tháng,mở cảnh báo nhập kế hoạch tháng
1,cảnh báo nhập kh tháng,mở cảnh báo nhập kế hoạch tháng
2,cảnh báo kh tháng,mở cảnh báo nhập kế hoạch tháng
3,cảnh báo tháng,mở cảnh báo nhập kế hoạch tháng
4,cảnh báo kế hoạch tháng,mở cảnh báo nhập kế hoạch tháng
...,...,...
285,bc kpi,mở báo cáo kpi chấm điểm
286,bc kpi,mở báo cáo kpi chấm điểm
287,bc kpi cđ,mở báo cáo kpi chấm điểm
288,mở báo cáo kpi chấm điểm,mở báo cáo kpi chấm điểm


In [63]:
results = deepcopy(datadata)
results['Dự đoán'] = [0]*len(results)
for i, ints in tqdm(enumerate(results['Câu lệnh sinh ra'])):
    output = encode_input_and_return_output(ints, db_df)
    results['Dự đoán'].loc[i] = output

290it [00:05, 51.82it/s]


In [ ]:
# results.to_csv("/content/drive/MyDrive/Achatbot/output/train_prediction_3.csv")

In [65]:
errors = results[results['Câu lệnh có sẵn']!=results['Dự đoán']]
errors['Câu lệnh có sẵn'].value_counts()

mở dữ liệu kế hoạch tháng                                        11
mở dữ liệu thực hiện ngày                                        10
mở dữ liệu thực hiện tháng                                       10
mở cảnh báo nhập kế hoạch năm                                     9
mở báo cáo chốt tháng (báo cáo chốt dữ liệu (ban chiến lược))     9
mở báo cáo chốt năm                                               4
mở cảnh báo nhập khách hàng quý                                   3
mở công thức chỉ tiêu                                             3
mở dữ liệu kế hoạch quý                                           2
mở báo cáo chi tiết n2 mới                                        2
mở báo cáo động                                                   2
mở báo cáo kpi chấm điểm                                          2
mở cảnh báo nhập kế hoạch tháng                                   1
Name: Câu lệnh có sẵn, dtype: int64

In [66]:
errors

,Câu lệnh sinh ra,Câu lệnh có sẵn,Dự đoán
5,kế hoạch tháng,mở cảnh báo nhập kế hoạch tháng,mở màn hình trình chiếu báo cáo tháng
22,cảnh báo kế hoạch quý,mở cảnh báo nhập khách hàng quý,mở dữ liệu kế hoạch quý
36,cb kh quý,mở cảnh báo nhập khách hàng quý,mở dữ liệu kế hoạch quý
37,cb quý,mở cảnh báo nhập khách hàng quý,mở dữ liệu kế hoạch quý
40,cảnh báo kh năm,mở cảnh báo nhập kế hoạch năm,mở dữ liệu kế hoạch năm
...,...,...,...
265,bc chốt năm,mở báo cáo chốt năm,mở dữ liệu kế hoạch năm
278,bc động,mở báo cáo động,mở quản lý báo cáo
281,bc động,mở báo cáo động,mở quản lý báo cáo
285,bc kpi,mở báo cáo kpi chấm điểm,mở báo cáo chăm sóc khách hàng


In [ ]:
# errors = results[results['Câu lệnh có sẵn']=="Mở dữ liệu KH tháng "]
# TH_error = errors[errors['Câu lệnh có sẵn']!=errors['Dự đoán']]
# TH_correct = errors[errors['Câu lệnh có sẵn']==errors['Dự đoán']]
# errors['Câu lệnh có sẵn'].value_counts()

In [ ]:
# TH_error.to_csv("/content/drive/MyDrive/Achatbot/output/KH_error.csv")

In [ ]:
# TH_correct.to_csv("/content/drive/MyDrive/Achatbot/output/KH_correct.csv")

In [ ]:
# errors.to_csv("/content/drive/MyDrive/Achatbot/output/train_prediction_error_3.csv")

# Đánh giá với độ chính xác Precision (Tổng số dự đoán đúng / Tổng số dự đoán)

In [ ]:
dictionary = {k: v for v, k in enumerate(database)}
preds = torch.tensor(results['Dự đoán'].apply(lambda x: dictionary[x]).tolist())
tags = torch.tensor(results['Câu lệnh có sẵn'].apply(lambda x: dictionary[x]).tolist())
precision = Precision(task="multiclass", average='micro', num_classes=55)
print(f"Độ chính xác (Precision) của mô hình: {round(precision(preds, tags).item()*100, 2)}")

Độ chính xác (Precision) của mô hình: 92.15


In [ ]:
dictionary = {k: v for v, k in enumerate(database)}
tags = torch.tensor(results['Câu lệnh có sẵn'].apply(lambda x: dictionary[x]).tolist())
count = 0
for i, inpu in tqdm(enumerate(results['Câu lệnh sinh ra'].tolist())):
    outpu = encode_input_and_return_top_n(inpu, db_df, 3).keys()
    if results['Câu lệnh có sẵn'].loc[i] in outpu:
        count += 1
print(count/(i+1))

2432it [00:29, 83.09it/s]

0.9917763157894737


# Demo

In [ ]:
inputt = "Bạn hãy mở lên dữ liệu kế hoạch tháng"
outputt = encode_input_and_return_top_n(inputt, db_df, 3)
outputt

{'mở dữ liệu kế hoạch tháng': 0.98,
 'mở dữ liệu thực hiện tháng': 0.93,
 'mở cảnh báo nhập kế hoạch tháng': 0.63}

In [ ]:
import json
with open("/content/drive/MyDrive/Achatbot/output/result.json", "w") as outfile:
    json.dump(outputt, outfile, indent=4)

In [ ]:
import gradio as gr

def image_classifier(Input):
    return encode_input_and_return_top_n(Input, db_df, 3)

demo = gr.Interface(fn=image_classifier, inputs="text", outputs="label")
# demo.launch(share=True)